In [1]:
import pandas as pd

In [2]:
path='C:/Users/dmckenzie/Downloads/tickets-April-19-2021-20_33.csv'

In [3]:
tickets=pd.read_csv(path,low_memory=False)

In [4]:
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from collections import defaultdict
import pyodbc 
import pandas as pd
import numpy as np
import umap
from sklearn import cluster
import nltk
def tokenlenfilter(tokenlist,minlen=2):
    tokens = [x for x in tokenlist if len(x)>minlen]
    return (tokens)

def lemmer(text):
    lemma_function = WordNetLemmatizer()

    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV

    return [lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in pos_tag(text)]

def get_ngrams(series: pd.Series,n:int):
    ngrams = series.copy().str.split(' ').explode()
    ngrams1 = series.copy().str.split(' ').explode()
    for i in range(1,n):
        foo=ngrams1.groupby(level=0).shift(-i)
        ngrams=ngrams.str.cat(foo,sep=' ')
    ngrams = ngrams.dropna()
    return pd.DataFrame(ngrams)

def get_multigrams(series,n):
    grams=get_ngrams(series,1)
    if n==1:
        return grams
    else:
        for i in range(1,n+1):
            if i==1:
                continue
            grams=grams.append(get_ngrams(series,i))
    return grams

In [5]:
from nltk.corpus import stopwords
stopWords = (stopwords.words('english'))
def tokennamefilter(tokenlist,minlen=2):
    tokens = [x for x in tokenlist if x not in stopWords+['dlrgroupcom','mailto','dlr','2019','2020','group','dlrgroupcomhttpwwwdlrgroupcom']]
    return (tokens)

In [6]:
def tokennamefilter2(tokenlist):
    tokens = [x for x in tokenlist if x in ['ordered','assigned','informed','phishing','install',
                                            'fixed','changed','inactivity','set','resolved','closing',
                                            'add','needed','updated','deleted','created','assigned','sent',
                                            'added','installed','reset','connected','deployed','shipped','reinstalled',
                                            'restarted','removed','reset','rebooted','ordered','onboarded']]
    return (tokens)

In [7]:
tickets=tickets[(tickets['Summary of Resolution'].isna()==False) & (tickets['Description'].isna()==False)]

In [8]:
tickets['action']=tickets['Summary of Resolution'].apply(nltk.word_tokenize).apply(tokennamefilter2).apply(' '.join).str.lstrip()

In [9]:
import re
def preprocessor(text): 
    text = re.sub(r'[^\w\s]','', text)
    text = text.lower()
    return text

In [10]:
from textblob import TextBlob
def getvp(tokenlist):
    tokens = [x for x,y in TextBlob(tokenlist).tags if 'V' in y]
    return (tokens)

In [11]:
tickets['cleansmry']=tickets['Summary of Resolution'].apply(preprocessor).str.lower().\
               apply(nltk.word_tokenize).\
               apply(tokennamefilter).apply(' '.join).str.lstrip()

In [12]:
tickets['cleandesc']=tickets['Description'].apply(preprocessor).str.lower().\
               apply(nltk.word_tokenize).\
               apply(tokennamefilter).apply(' '.join).str.lstrip()

In [13]:
tickets['verbs']=tickets['cleansmry'].apply(getvp).apply(' '.join).str.lstrip()

In [14]:
tickets['verbs'].value_counts().head(50)

               4997
installed       647
added           419
resolved        398
updated         260
add             200
created         180
reinstalled     159
deployed        154
informed        124
fixed           103
needed           95
sent             95
phishing         86
changed          84
set              65
reset            60
restored         60
moved            60
see              59
working          54
removed          53
deleted          50
install          49
gave             49
using            48
assigned         47
rebooted         43
restarted        43
switched         36
closing          35
provided         34
replaced         34
found            32
remove           30
repaired         30
applied          28
connected        28
update           27
adding           27
shipped          27
ordered          26
used             24
offboarded       24
received         23
closed           22
onboarded        22
worked           22
rolled           21
use              21


In [15]:
tickets2=tickets[tickets['action'].apply(nltk.word_tokenize).apply(len)==1]

In [16]:
tickets2=tickets2[(tickets2['cleandesc']!='') & (tickets2['cleansmry']!='') & 
                  (tickets2['cleansmry']!=' ') & (tickets2['cleandesc']!=' ')]

In [17]:
len(tickets2)

4225

In [18]:
tickets2['action2']=tickets2['action']

In [19]:
tickets2=tickets2.set_index('action2')

In [20]:
ADdf=get_ngrams(tickets2[tickets2['action']!=""]['cleandesc'],1).\
    reset_index().\
    query('cleandesc!=""').\
    assign(dummy=1).\
    groupby(['action2','cleandesc'])['dummy'].sum().reset_index().sort_values(['action2','dummy'],ascending=False)

In [31]:
get_ngrams(tickets2[tickets2['action']!=""]['cleandesc'],1).\
    reset_index().\
    query('cleandesc!=""').\
    assign(Count=1).\
    groupby(['action2','cleandesc'])['Count'].sum().reset_index().sort_values(['action2','Count'],ascending=False)\
    .groupby(['action2']).head(1).rename(columns={'action2':'Resolution Verb','cleandesc':'Intake Desccription'})

,Resolution Verb,Intake Desccription,Count
45527,updated,update,150
42422,shipped,laptop,20
40259,set,architecture,67
38175,sent,access,62
37485,restarted,find,20
35085,resolved,find,152
33221,reset,password,49
32079,removed,please,37
30366,reinstalled,bluebeam,45
29692,rebooted,computer,21


In [379]:
ADdf

,action2,cleandesc,dummy
44785,updated,update,150
43916,updated,need,114
43181,updated,find,111
44794,updated,us,110
43086,updated,engineering,108
...,...,...,...
2396,add,yorks,1
2397,add,yough,1
2398,add,youre,1
2399,add,zans,1


In [ ]:
tops=adf.\
    drop_duplicates(subset=['RoomId','Cluster']).\
    assign(dummy=1).\
    groupby(['Label2','Label','Cluster'])['dummy'].\
    sum().\
    reset_index().\
    sort_values(['Label2','dummy'],ascending=False).\
    set_index(['Label2']).\
    join(RC2.drop_duplicates(subset=['RoomId']).\
         assign(RoomTypeTotal=1).\
         groupby(['Label2'])['RoomTypeTotal'].\
         sum().\
         reset_index().\
         set_index(['Label2'])).\
    reset_index().\
    assign(RoomClusterPorp=lambda x: round((x.dummy / x.RoomTypeTotal),2)*100).\
    set_index(['Cluster']).\
    join(RC2[RC2['Cluster']!='a-1'].\
    drop_duplicates(subset=['RoomId','Cluster']).\
    assign(ClustTotal=1).\
    groupby(['Cluster'])['ClustTotal'].\
    sum().reset_index().\
    sort_values(['ClustTotal'],ascending=False).\
    set_index(['Cluster'])).\
    reset_index().\
    assign(PopTotal=lambda x: round((x.ClustTotal/len(RC2[RC2['Cluster']!='a-1'].drop_duplicates(subset=['RoomId']))),2)*100).\
    assign(Delta=lambda x: x.RoomClusterPorp-x.PopTotal).\
    sort_values(['Label2','Delta'],ascending=False).\
    groupby(['Label2']).\
    head(10).set_index(['Cluster','Label2']).join(topitem[['Cluster','Name','Label2']].set_index(['Cluster','Label2'])).reset_index()

In [106]:
#tickets[tickets['action']=='installed']

In [108]:
get_ngrams(tickets[tickets['action']=='deleted']['cleansmry'],1).\
                value_counts().head(25)

cleansmry   
deleted         67
                44
folder          11
file            11
appdata         11
ost             10
files            7
20               6
error            5
found            4
account          4
copied           4
box              3
items            3
apps             3
localappdata     3
location         3
renamed          3
data             3
causing          2
calendar         2
restored         2
one              2
outlook          2
opened           2
dtype: int64

In [12]:
get_ngrams(tickets['Summary of Resolution'].str.lower().\
               apply(nltk.word_tokenize).\
               apply(tokennamefilter).apply(' '.join).str.lstrip(),2).\
                value_counts().head(20)

Summary of Resolution
bim 360                  354
self resolved            249
add user                 237
user project             152
closing due              150
informed user            150
added user               134
cc license               132
added project            106
https :                  102
dell command             102
project .                 99
360 project               98
user bim                  89
issue resolved            84
sent user                 81
deployed pdq              80
dd :                      77
due response              63
tn :                      61
dtype: int64

In [124]:
get_ngrams(tickets[(tickets['Summary of Resolution'].isna())==False]['Summary of Resolution'].str.lower().\
               apply(nltk.word_tokenize).\
               apply(tokennamefilter).apply(' '.join).str.lstrip(),3).\
                value_counts().head(20)

Summary of Resolution 
add user project          124
bim 360 project            98
user bim 360               89
added user bim             82
closing due response       54
closing due inactivity     50
msi gs66 -                 46
sent welcome email         44
added bim 360              43
added project .            38
dell command updates       38
ran dell command           34
closing due stagnation     32
updated latest version     26
360 project .              25
sent user instructions     24
dell command update        24
get sent welcome           24
issue resolved .           23
let user know              21
dtype: int64

In [126]:
get_ngrams(tickets[tickets['Verbs']!='']['Verbs'],1).value_counts().head(40)

Verbs      
installed      1026
added           576
resolved        481
deployed        327
updated         318
created         291
add             289
needed          200
set             195
sent            179
reinstalled     173
fixed           166
informed        164
changed         154
working         138
using           131
ordered         120
restarted       119
shipped         114
removed         113
phishing        108
moved           107
deleted         104
assigned        102
gave             97
see              94
reset            87
restored         82
connected        81
worked           81
closing          79
install          79
found            78
rebooted         78
get              69
go               66
opened           65
went             63
bim              63
got              60
dtype: int64